# ToDo: Unit Tests
Some testing is in here but full unit tests are required for each function

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
dbutils.widgets.text("model_name", "users.brian_mcconnell.dbdemos_customer_churn")

In [0]:
%pip install weasyprint mlflow mdtex2html graphviz

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
%restart_python

In [0]:
import logging
from databricks.sdk import WorkspaceClient
import mlflow

import sys, os
sys.path.append(os.path.abspath('./tmpl'))  # or the appropriate relative/absolute path

import tmpl

from mrm_utils import *
from mrm_objects import *
from mrm import *

import json
import requests
import time

from weasyprint import HTML

logger = logging.getLogger('databricks')

a_bit_of_space = '<div class="col-xs-12" style="height:100px;"></div>'
a_little_bit_of_space = '<div class="col-xs-12" style="height:70px;"></div>'

mlflow.set_registry_uri('databricks-uc')
model_name=dbutils.widgets.get("model_name")


In [0]:
%run ./MRM-Functions

# extract_run_libraries

In [0]:
#def extract_run_libraries(run_tags):


# extract_run_cluster

In [0]:
#def extract_run_cluster(run_tags):



# extract_run_data_sources

In [0]:
#def extract_run_data_sources(run_object, run_tags):



# extract_run_artifact_signature

In [0]:
#def extract_run_artifact_signature(fields):



# extract_run_artifact_flavors

In [0]:
#def extract_run_artifact_flavors(flavors):


# extract_run_artifacts

In [0]:
#def extract_run_artifacts( run_tags):


# extract_source_lineage_name

In [0]:
#def extract_source_lineage_name(data_source):


# extract_notebook

In [0]:
#def extract_notebook(html_content):


# process_lineage

In [0]:
#def process_lineage( response):


# process_model_parent

In [0]:
#def process_model_parent(response):


# process_model_run

In [0]:
#def process_model_run( response, run_id):
#

# process_model_versions

In [0]:
#def process_model_versions(data, model_name, model_parent_aliases):


# process_model_version

In [0]:
#def process_model_version(data, model_version):


# get_model_version

In [0]:
#def get_model_version(model_name, model_version):



# get_model_versions


In [0]:
#def get_model_versions( model_name, model_parent_aliases):


# get_model_parent

In [0]:
#def get_model_parent( model_name):


# get_model_run

In [0]:
#def get_model_run(run_id):


# get_lineage_rec

In [0]:
#def get_lineage_rec(upstream_source, upstream_type):


# get_lineage

In [0]:
#def get_lineage(data_source_names, model_name):



# get_notebook_from_job



In [0]:
#def get_notebook_from_job( job_id):



# get_notebook

In [0]:
#def get_notebook(remote_path):


# generate_doc

In [0]:

model_version=1
output_file=None
verbatim_file=None
X="""
        Public entry point for model risk management PDF output. Given a model name, an optional model version and a
        target output file, we will fetch all the required information from various databricks API, bring that
        technical and business context together and generate PDF output accordingly. After multiple consideration
        being the use of e.g. LateX library, we decided to leverage HTML as main format as it supports markdown
        information, HTML that we can "beautify" using boostrap CSS and convert to PDF document.
        :param model_name: the name of the model to fetch from databricks

        :param model_version: the version of the model to fetch from databricks
        :param output_file: the output file to write PDF document if omitted then it will only be displayed in the cell (the default)
        :param verbatim_file: giving user the opportunity to supply their own verbatim files instead of default
        :return:
"""


# load our text
verbatim = load_verbatim(verbatim_file)

# retrieve model from Unity Catalog
model_parent = get_model_parent(model_name)

model_versions = get_model_versions(model_name, model_parent.model_parent_aliases)

model_version = get_model_version(model_name, model_version)
        
#model_run = get_model_run(model_version.model_run_id)




In [0]:
displayHTML(model_parent.to_html(h_level=1))



In [0]:
displayHTML(model_versions.to_html(h_level=1))

In [0]:
displayHTML(model_version.to_html(h_level=1))


In [0]:
#model_run = get_model_run()
w = WorkspaceClient()
response = w.experiments.get_run("89d2eec06faf4490959a6c0ce6d771f8")
data=response.as_dict()
#process_model_run(data, run_id)
run_object = data['run']
run_data = run_object['data']
run_info = run_object['info']
run_tags = key_value_to_dict(run_data['tags'])

run_experiment_id = run_info['experiment_id']
run_timestamp = parse_time(run_info['start_time'])
run_parent_id = run_tags.get('mlflow.parentRunId') or None
run_description = run_tags.get('mlflow.note.content') or None
run_user = run_tags.get('mlflow.user') or None
run_workspace = run_tags.get('mlflow.databricks.workspaceURL') or None

run_data_sources = extract_run_data_sources(run_object, run_tags)
run_cluster = extract_run_cluster(run_tags)
run_artifacts = extract_run_artifacts(run_tags)
run_libraries = extract_run_libraries(run_tags)

In [0]:
if run_description:
    run_description = ExperimentDescription(run_description)

In [0]:
if 'mlflow.source.type' in run_tags.keys():
            source_type = run_tags.get('mlflow.source.type')
            if source_type == 'NOTEBOOK':
                # Pull associated notebook
                # TODO: use revision ID
                source_name = run_tags.get('mlflow.source.name')
                source_code = get_notebook(source_name)
                source_commit = run_tags.get('mlflow.databricks.notebookRevisionID')
            elif source_type == 'JOB':
                # Pull associated JOB output
                source_name = run_tags.get('mlflow.source.name')
                source_code = get_notebook_from_job(source_name)
                source_commit = None
            else:
                source_name = None
                source_code = None
                source_commit = None

            if 'mlflow.databricks.gitRepoUrl' in run_tags:
                source_url = run_tags.get('mlflow.databricks.gitRepoUrl')
                source_commit = run_tags.get('mlflow.databricks.gitRepoCommit')
            else:
                source_url = None
else:
            source_type = None
            source_name = None
            source_commit = None
            source_url = None
            source_code = None

In [0]:
get_notebook_from_job(source_name)

In [0]:
source_name = run_tags.get('mlflow.source.name')


In [0]:
source_name

In [0]:
run_id = source_name.split('/')[-1]
w = WorkspaceClient()

response = w.jobs.export_run(run_id)
data=response.as_dict()

#        response = json.loads(requests.get(url=url, headers=headers).text)

content = list(filter(lambda x: x['type'] == 'NOTEBOOK', data['views']))




In [0]:
if len(content) > 0:
    html_org_content = content[0]['content']
    #displayHTML(html_org_content)

In [0]:
displayHTML(html_org_content)

In [0]:

    
yy=extract_notebook(html_org_content)


In [0]:
notebook_regex = re.compile(
    "DATABRICKS_NOTEBOOK_MODEL = '((?:[A-Za-z0-9+/]{4})*(?:[A-Za-z0-9+/]{2}==|[A-Za-z0-9+/]{3}=)?)'"
)
matches = notebook_regex.findall(html_org_content)
if matches:
    qq=Notebook(matches[0])

In [0]:
displayHTML(qq.to_html())


In [0]:
if 'params' in run_data:
            run_params = ExperimentParameters(key_value_to_dict(run_data['params']))
else:
            run_params = {}

if 'metrics' in run_data:
            run_metrics = ExperimentMetrics(key_value_to_dict(run_data['metrics']))
            # TODO: consider extracting metrics across runs or find way to vizualize in parallel graph
else:
            run_metrics = {}

YY=ModelExperiment(
            run_id,
            run_description,
            run_user,
            run_workspace,
            run_experiment_id,
            run_timestamp,
            run_params,
            run_metrics,
            run_data_sources,
            run_parent_id,
            source_type,
            source_name,
            source_url,
            source_commit,
            source_code,
            run_cluster,
            run_artifacts,
            run_libraries
        )

In [0]:
data

In [0]:
# retrieve model data input and lineage
if model_run.run_data_sources:
    data_sources = model_run.run_data_sources
    data_lineage = None #get_lineage(data_sources.sources(), model_name)
else:
    data_sources = None
    data_lineage = None

model_parent.to_html(h_level=1)

if model_parent.model_description:
    model_parent.model_description.to_html(h_level=2))

model_versions.to_html(h_level=1)

model_version.to_html(h_level=1)


if model_version.model_description:
    model_version.model_description.to_html(h_level=2)

model_run.to_html(h_level=1)
if model_run.run_description:
    model_run.run_description.to_html(h_level=2)
if model_run.run_artifacts:
            html.extend(model_run.run_artifacts.to_html(h_level=3))


if model_run.source_code:
        model_run.source_code.to_html(h_level=2)


if model_run.run_params:
    model_run.run_params.to_html(h_level=3)

if model_run.run_libraries and model_run.run_libraries.non_empty():
    html.extend(model_run.run_libraries.to_html(h_level=2))

if model_run.run_artifacts:
    for artifact in model_run.run_artifacts.artifacts:
        if artifact.signature:
            artifact.signature.to_html(h_level=3)

if data_sources:
    data_sources.to_html(h_level=2)

if data_lineage:
    data_lineage.to_html()